# Making prediction

In [ ]:
from utils.bbox import *
from utils.colors import *
from utils.image import *
from utils.utils import *
from utils.modelling import *
from keras.models import load_model
from tensorflow.python.util import deprecation
import matplotlib.pyplot as plt
import ipywidgets as widgets
import torch
import torchvision
import torchvision.transforms.functional as TF
import PIL
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Load model & Declare constants

In [ ]:
# load yolov3 model
model = load_model('yolov3_model.h5')
# define the expected input shape for the model
input_w, input_h = 416, 416
# define the anchors
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]
# define the probability threshold for detected objects
class_threshold = 0.6
# define the labels
labels = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck",
          "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench",
          "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
          "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard",
          "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
          "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana",
          "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
          "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse",
          "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
          "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]

In [ ]:
def predictTransformedImage(input_image, shape, model, anchors, class_threshold, labels):
    # convert to numpy array
    image_w, image_h = input_image.size
    resized_image = TF.resize(input_image, shape)
    image = img_to_array(resized_image)
    # scale pixel values to [0, 1]
    image = image.astype('float32')
    image /= 255.0
    yhat = model.predict(np.expand_dims(image, 0))
    boxes = list()
    for i in range(len(yhat)):
        # decode the output of the network
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
    # correct the sizes of the bounding boxes for the shape of the image
    correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
    # suppress non-maximal boxes
    do_nms(boxes, 0.5)
    # get the details of the detected objects
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
    # summarize what we found
    for i in range(len(v_boxes)):
        print(v_labels[i], v_scores[i])

    # plot the image
    plt.imshow(input_image)
    # get the context for drawing boxes
    ax = plt.gca()
    # plot each box
    for i in range(len(v_boxes)):
        box = v_boxes[i]
        # get coordinates
        y1, x1, y2, x2 = box.ymin, box.xmin, box.ymax, box.xmax
        # calculate width and height of the box
        width, height = x2 - x1, y2 - y1
        # create the shape
        rect = Rectangle((x1, y1), width, height, fill=False, color='white')
        # draw the box
        ax.add_patch(rect)
        # draw text and score in top left corner
        label = "%s (%.3f)" % (v_labels[i], v_scores[i])
        plt.text(x1, y1, label, color='white')
    # show the plot
    plt.show()

## Load image

In [ ]:
# define our new photo
photo_filename = './images/dog.jpg'
# load and prepare image
image, image_w, image_h = load_image_pixels(photo_filename, (input_w, input_h))
# make prediction
yhat = model.predict(image)
# summarize the shape of the list of arrays
print([a.shape for a in yhat])

boxes = list()
for i in range(len(yhat)):
    # decode the output of the network
    boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, input_h, input_w)
# correct the sizes of the bounding boxes for the shape of the image
correct_yolo_boxes(boxes, image_h, image_w, input_h, input_w)
# suppress non-maximal boxes
do_nms(boxes, 0.5)
# get the details of the detected objects
v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)
# summarize what we found
for i in range(len(v_boxes)):
    print(v_labels[i], v_scores[i])

show_boxes(photo_filename, v_boxes, v_labels, v_scores)

# Widget application

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from IPython.core.display import clear_output

## Widget 1: show original image

PyTorch transformation function list: [link](https://pytorch.org/docs/stable/torchvision/transforms.html)

PIL image processing list: [link](https://pytorch.org/docs/stable/torchvision/transforms.html)

In [ ]:
# Widget features declaration
output_imageOriginal = widgets.Output()
output_imageOriginal.layout.border = '1.0px solid'
output_imageOriginal.layout.width = 'calc(100% - 14ex)'
input_imagePath = widgets.Text(value='./images/dog.jpg', description='Image path')
button_imageShow = widgets.Button(description='Show image', button_style='', tooltip='Click to show image')
imageOriginal = PIL.Image.open(input_imagePath.value)
# Widget function definition
def on_imageShowBtn_clicked(_):
    global imageOriginal
    with output_imageOriginal:
        clear_output()
        imageOriginal = PIL.Image.open(input_imagePath.value)
        print("Image size: {}".format(imageOriginal.size))
        display(imageOriginal)

button_imageShow.on_click(on_imageShowBtn_clicked)

# Grouping widgets
box_widget1 = widgets.VBox([ input_imagePath, button_imageShow, output_imageOriginal ])

## Widget 2: rotate tab

In [ ]:
# Widget features declaration
slider_rotateAngle = widgets.IntSlider(
    value=0, min=-180, max=180, step=1,
    description='Angle:',
    orientation='horizontal',
    readout=True, readout_format='d')
button_imageRotate = widgets.Button(description='Rotate', button_style='', tooltip='Click to rotate image')
output_imageRotated = widgets.Output()
output_imageRotated.layout.border = '1.0px solid'
output_imageRotated.layout.width = 'calc(100% - 14ex)'


# Widget function definition
def on_imageRotateBtn_clicked(_):
    global imageRotated
    with output_imageRotated:
        clear_output()
        imageRotated = TF.rotate(
            imageOriginal, slider_rotateAngle.value, 
            resample=PIL.Image.NEAREST, expand=True)
        print("Rotated image size: {}".format(imageRotated.size))
        display(imageRotated)

button_imageRotate.on_click(on_imageRotateBtn_clicked)

# Grouping widgets
box_widget2 = widgets.VBox([slider_rotateAngle, button_imageRotate, output_imageRotated ])
# box_widget2.layout.height = 'calc(50%)'

In [ ]:
# rotate tab
tab_rotate = widgets.HBox([ box_widget1, box_widget2 ])
tab_rotate

## Widget 3: crop tab

In [ ]:
# rotate tab
tab_crop = widgets.HBox([ box_widget1, box_widget3 ])
tab_crop

In [ ]:
# Widget features declaration
slider_cropt = widgets.IntSlider(
    value=0, min=0, max=imageOriginal.size[1], step=1,
    description='Top:', orientation='horizontal', readout=True, readout_format='d')
slider_cropl = widgets.IntSlider(
    value=0, min=0, max=imageOriginal.size[0], step=1,
    description='Left:', orientation='horizontal', readout=True, readout_format='d')
slider_cropw = widgets.IntSlider(
    value=imageOriginal.size[0], min=slider_cropl.value, max=imageOriginal.size[0], step=1,
    description='Width:', orientation='horizontal', readout=True, readout_format='d')
slider_croph = widgets.IntSlider(
    value=imageOriginal.size[1], min=slider_cropt.value, max=imageOriginal.size[1], step=1,
    description='Height:', orientation='horizontal', readout=True, readout_format='d')
box_crop1 = widgets.HBox([ slider_cropl, slider_cropt ])
box_crop2 = widgets.HBox([ slider_cropw, slider_croph ])
box_crop = widgets.VBox([ box_crop1, box_crop2 ])
button_crop = widgets.Button(description='Crop', button_style='', tooltip='Click to crop image')
output_imageCropped = widgets.Output()
output_imageCropped.layout.border = '1.0px solid'
output_imageCropped.layout.width = 'calc(100% - 14ex)'

# Widget function definition
def on_imageCropBtn_clicked(_):
    global imageCropped
    with output_imageCropped:
        clear_output()
        # imageCropped = TF.center_crop(imageOriginal, (slider_croph.value, slider_cropw.value))
        imageCropped = TF.crop(imageOriginal, slider_cropt.value, slider_cropl.value, slider_croph.value, slider_cropw.value)
        print("Rotated image size: {}".format(imageCropped.size))
        display(imageCropped)

button_crop.on_click(on_imageCropBtn_clicked)

# Grouping widgets
box_widget3 = widgets.VBox([box_crop, button_crop, output_imageCropped ])# Widget features declaration
slider_cropSize = widgets.IntSlider(
    value=0, min=-180, max=180, step=1,
    description='Angle:',
    orientation='horizontal',
    readout=True, readout_format='d')
button_imageRotate = widgets.Button(description='Rotate', button_style='', tooltip='Click to rotate image')
output_imageRotated = widgets.Output()
output_imageRotated.layout.border = '1.0px solid'
output_imageRotated.layout.width = 'calc(100% - 14ex)'


# Widget function definition
def on_imageRotateBtn_clicked(_):
    global imageRotated
    with output_imageRotated:
        clear_output()
        imageRotated = TF.rotate(
            imageOriginal, slider_rotateAngle.value, 
            resample=PIL.Image.NEAREST, expand=False)
        print("Rotated image size: {}".format(imageRotated.size))
        display(imageRotated)

button_imageRotate.on_click(on_imageRotateBtn_clicked)

# Grouping widgets
box_widget2 = widgets.VBox([slider_rotateAngle, button_imageRotate, output_imageRotated ])
box_widget2.layout.height = 'calc(50%)'

In [ ]:
# defining a list with the contents of our windows
children = [tab_rotate, tab_crop]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'rotate')
tab.set_title(1, 'crop')
tab

In [ ]:
predictTransformedImage(imageRotated, (input_w, input_h), model, anchors, class_threshold, labels)

In [ ]:
predictTransformedImage(imageCropped, (input_w, input_h), model, anchors, class_threshold, labels)

## Putting all widgets together